Энэ удаагийн бие даалтын ажлаар 2 давхаргат LSTM (long-short term memory) ашиглан мэдээний төрөл ангилах модел сургах болно.
LSTM нь RNN(recurren neural network) - ын нэгэн төрөл бөгөөд, дарааллаас хамаарсан sequence - ыг хэрхэн таамаглахыг суралцах чадвартай. Уг суралцах чадварыг нь машин орчуулга, яриа танилт гэх мэт салбаруудад ашигладаг.
Бүх төрлийн RNN нь recurrent давхрагатаа feedback loop (буцах давтамж) - тай байдаг. Тэдгээр нь тодорхой хугацааны турш мэдээллийг хадгалж өгч байдаг. Харин энгийн RNN - ын алдааны функцын градент нь хугацааны турш алга болсоор байдаг учир урт хугацааны хамааралтай асуудлыг шийдвэрлэж чаддаггүй.
Харин LSTM нь өөртөө нэмэлтээр 'memory cell' гэх урт хугацааны турш мэдээллийг хадгалах санах ой төрлийн unit-тэй. Мөн мэдээллийг memory cell-д хэзээ оруулах, хэзээ нь мартуулахыг удирддаг хаалгануудтай байдаг. Энэ архитехтурыг ашиглан LSTM ын урт хугацааны туршид чухал мэдээллээ мартахгүй хадгалж чаддаг.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import re
import os
import time
import torch                                          # pytorch нь python-ы машин сургалтын library
import torch.nn as nn                                 # pytorch - ын бүх neural network - ын эх класс
import torch.nn.functional as F                       # nn - ын модулиуд нь layer болж ажилладаг бол functional ынх классууд нь арифметик үйлдэл хийдэг функцууд байна
from torch.nn.utils.rnn import pack_padded_sequence   # pad хийсэн tensor ыг багцалж, lstm - д өгөхөд бэлэн болгон
import torch.utils.data as data                       # pytorch - ын дататай харьцах классуудын эх класс
from torch.utils.data import DataLoader               # DataLoader module
from torch.utils.data.sampler import RandomSampler    # random өгөгдөл сонгож буцаах нэгэн төрлийн Sampler
import pandas as pd                                   # Pandas нь өгөгдлийг DataFrame төрөлд оруулж, түүнтэй ажиллах боложмийг олгоно
from collections import Counter                       # Их хэмжээний өгөгдөл тоолох

## Дата бэлтгэл
&nbsp;&nbsp;&nbsp;&nbsp;Pytorch - ын Dataset модулийг ашиглах учир датагаа түүндээ зориулж бэлдэх хэрэгтэй. Иймээс Numpy - ын бүтэцтэй өгөгдөл биш, tensor өгөгдөл ашиглана. Tensor нь pytorch-ын олон хэмжээст өгөгдөл(нэг төрлийн) хадгалагч матриц юм. Pytorch - д 10 төрлийн, CPU болон GPU - д зориулсан tensor-ууд бий. Pytorch нь Numpy массивтай төстэй бүтэцтэй tensor-уудыг ашиглан өгөгдлөө сургалтын үед GPU рүү шийлжүүлэх боломжийг олгодог.
    

Dataset класс үүсгэх

In [ ]:
class ClassificationDataset(data.Dataset):
    def __init__(self, data_path, vocab,classes):                                   # Dataset обьект анх үүсэх үед дуудагдах функц
        self.word2id = vocab
        self.class2id = classes
        self.txt_seqs = pd.read_csv(data_path, encoding='utf-8', engine='python')   # өгөгдсөн байршил дахь .csv файлыг уншин, dataframe үүсгэнэ
        self.num_class = len(self.class2id)
        print("Loaded Dataset", len(self.txt_seqs))

    def __len__(self):                                                              # Dataset ын уртыг буцаана (DataLoader үүсэх үед)
        return len(self.txt_seqs)

    def __getitem__(self, index):                                                   # dataset - с өгөгсдөн index дахь датаг буцаана
        data = self.txt_seqs.iloc[index]
        word_id_seq = self._preprocess(data["news"])
        return word_id_seq, torch.LongTensor([self.class2id.get(data[" label"])])   # оролтын болон target дата хослолыг буцаана

    def _preprocess(self, txt_seq):
        return torch.LongTensor([self.word2id.get(token, self.word2id['<unk>']) for token in txt_seq.split()])  # оролтын өгөгдлийг цэвэрлэж, token (index) руу хөрвүүлэн, tensor утга болгоно 

&nbsp;&nbsp;&nbsp;&nbsp;Одоо датагаа багц багцаар (batch) уншихад бэлтгэе. Датаг batch уудад хувааж, сургалтанд ашиглахад зориулсан DataLoader класс бий. DataLoader обьект (iterable) нь өгөгдсөн dataset-р гүйж давтана. Доорх BatchSampler класс нь өгөгдсөн датасетээс датаг random оор сонгож өгнө. Цаашид бид мөн сургалтын явцад validate хийх датагаа batch, batch аар нь __iter__() - р авах болно.

In [ ]:
class BatchSampler(object):
    def __init__(self, data_source, batch_size):
        self.sampler = RandomSampler(data_source)                   # random өгөгдөл сонгож буцаах нэгэн төрлийн Sampler
        self.batch_size = batch_size
        self.random_batches = self._make_batches()                  # санамсаргүйгээр өгөгдлүүдийг сонгож, batch үүсгэх

    def _make_batches(self):
        indices = [i for i in self.sampler]
        batches = [indices[i:i + self.batch_size] for i in range(0, len(indices), self.batch_size)]
        random_indices = torch.randperm(len(batches)).tolist()
        return [batches[i] for i in random_indices]                 # batch буцаана

    def __len__(self):
      return len(self.sampler)

    def __iter__(self):
        for batch in self.random_batches:                           # санамсаргүй сонгогдсон batch уудыг нэг нэгээр нь дуудаж ашиглана
          if len(self.random_batches) > 1:
            yield batch

LSTM, CNN, RNN гэх мэт network - уудад адил урттай sequential оролтыг өгөх шаардлагатай байдаг. Тиймээс өөр өөр урттай оролтуудыг 'pad' хийж адил урттай болгоё.
Ж/нь: <div>[1,2,3,4,5,6]<br>[1,2,3]<br>[1,2,3,4,5]</div>
Гэсэн batch хэмжээ нь 3 оролтыг:
<div>[1,2,3,4,5,6]<br>[0,0,0,1,2,3]<br>[0,1,2,3,4,5]</div>
болгон адил урттай болгох юм.

Энэ нь pre-padding гэх арга бөгөөд, post-padding - ыг ашиглаагүйн шалтгаан нь, padding нь sequence нь төгсгөлд байрлах юм бол sequence ын эхэнд байсан чухал үгүүд мартагдах болно.

In [ ]:
class Padder(object):
    def __init__(self):pass
    def __call__(self, batch):
        input_seqs, label_seqs = zip(*batch)                      # Batch өгөгдлийг оролт/гаралт - аар задлав
        lengths = [len(seq) for seq in input_seqs]                # Batch дахь оролтын өгөгдлүүдийн уртууд
        input_padded_seqs = torch.zeros(len(input_seqs), max(lengths)).long()     # Уртуудаас хамгийн уртаар нь хоосон tensor үүсгэв.  shape = batch_size x хамгийн урт оролтын урт
        output_padded_seqs = torch.zeros(len(input_seqs)).long()  # Мөн адил гаралтад зориулсан хоосон tensor үүсгэв. shape = batch_size
        for i, input in enumerate(input_seqs):
            end = lengths[i]
            input_padded_seqs[i, (max(lengths) - end):] = input[:end]   # Дээр үүсгэсэн хоосон tensor-тоо орлтын утгуудаа төгсгөл хэсэгт нь хийв
            output_padded_seqs[i] = label_seqs[i][0]              
        return input_padded_seqs, torch.IntTensor(lengths), output_padded_seqs

In [ ]:
def build_data_loader(data, vocab, classes, batch_size, num_workers):
    dataset = ClassificationDataset(data, vocab, classes)              # Датасет обьект үүсгэв
    batch_sampler = BatchSampler(dataset,batch_size)                   # dataset - с өгөгдлийг багцлах обьектоо үүсгэв
    collate_fn = Padder()                                              # pre-padding обьект
    data_loader = DataLoader(dataset, batch_sampler=batch_sampler, collate_fn=collate_fn, num_workers=num_workers) # DataLoader нь датасетээс batch-ууд үүсгэж, pad хийнэ
    return data_loader                                                 # DataLoader нь pytorch-ын дататай харьцах зүрх нь болсон модуль юм.

## Neural network архитехтур

Pytorch - ын бүх neural network модулиудын эх класс нь torch.nn.Module байдаг.

In [ ]:
class LstmClassification(nn.Module):
    def __init__(self, num_embeddings, embedding_dim,hidden_size, num_layers,num_class, dropout, bidirectional):
        super(LstmClassification, self).__init__()
        self.embedding = nn.Embedding(num_embeddings, embedding_dim)      # Embedding нь үгсийн утгыг тоон хэлбэрт хөрвүүлж, олон хэмжээст вектор болгон хадгалах үүрэгтэй.
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers,batch_first=True,dropout=dropout,bidirectional=bool(bidirectional))  # 2 чиглэлт, LSTM үүсгэлт
        fc_in_dim = hidden_size * 2 * num_layers if bidirectional else hidden_size     # BiLSTM(Bidirecional LSTM) ын гаралт нь ердийн LSTM - с 2 чиглэлтэй
        # учир 2 дахин урт хэлбэртэй гаралт гарна, LSTM - ын дараагаар бид fully connected layer-лүү өгөгдлөө оруулах учир fc-ын оролтын хэмжээсийг тодорхойлж өгнө
        self.linearDP = torch.nn.Dropout(dropout)    # LSTM нь сургалтын өгөгдөлд overfit (хэт итгэлтэй болох) магадлалтай тул, сургалтын явцад таамгаар оролтуудыг хасан, accuracy - г нэмэгдүүлнэ.
        self.fc = nn.Linear(fc_in_dim, num_class)    # fully connected layer нь оролтоо дараагийн layer - т шилжүүлэхэд хэлбэржүүлж, тусална.
        
    def forward(self, padded_input, input_lengths):
        padded_input = self.embedding(padded_input) # оролтын өгөгдлүүдийг embedding vector луу шилжүүлэв. shape = batch_size * max(input_lengths) * embedding_dim
        packed_input = pack_padded_sequence(padded_input, input_lengths.cpu(),batch_first=True, enforce_sorted=False)   # lstm руу өгөгдлөө оруулахын өмнө pad хийсэн sequence - үүдээ боох буюу pack лах ёстой байдаг.
        _, (hidden, _) = self.lstm(packed_input)   # lstm нь гаралтын утга, оролтын утгын hidden state, cell-ын state ыг буцаадаг
        hidden = torch.reshape(hidden.permute(1, 0, 2), (hidden.shape[1], -1))  # lstm - с гарсан hidden state ын хэлбэрийг өөрчилж, fc - д өгсөнөөр тухайн batch - ыг таамгууд гарна. 
        score = self.fc(self.linearDP(hidden))   # batch ын prediction - ууд. shape = batch_size * max(input_lengths)
        return score

In [ ]:
class Solver(object):           # Solver класс нь сургалтын явцыг удирдах болно.
    def __init__(self, data, model, criterion, optimizer, epochs, max_norm, print_freq, file_path, use_cuda,n_classes):
        self.tr_loader = data['tr_loader']      # сургалтын dataloader
        self.cv_loader = data['cv_loader']      # validation dataloader
        self.model = model                      # LSTMClassification модель
        self.criterion = criterion              # CrossEntropyLoss
        self.optimizer = optimizer              # Adam optimizer
        self.n_classes = n_classes              # гаралтын утгын төрлийн тоо

        self.use_cuda = use_cuda                # сургалтын үед тооцоолол хийх tensor утгуудаа gpu дээр ажиллуулах үгүйг шийднэ.
        self.epochs = epochs                    # нийт датасетийг хэдэн удаа давтаж сургахыг удирдана.
        self.max_norm = max_norm                # gradient clip хийх хамгийн мах утга
        self.file_path = file_path              # model хадгалах байршил
        self.print_freq = print_freq            # хэвлэж харуулах давтамж
        self.traini = 0                         # сургалтын явцыг бүртгэнэ
        self.evali = 0
        self.prev_val_loss = float("inf")       # өмнөх loss ын утгыг хадгална
        self.best_val_loss = float("inf")       # хамгийн бага loss ын утгыг хадгална. Хэрэв үүнээс бага loss гарвал моделоо хадгална.

    def train(self):
        for epoch in range(self.epochs):        # epochs дуустал давтана.
            print("Training...")
            self.model.train()                  # dropout болон BatchNorm - ыг идэвхжүүлж, нэг ёсондоо моделоо сургалтын горимд шилжүүлнэ. 
            start = time.time()

            tr_avg_loss, epoch_acc, epoch_weighted_recall = self._run_one_epoch(epoch)  # нэг epoch сургах явц буюу сургалтын датаг бүтнээр нь нэг удаа дуустал сургана.

            print('-' * 85)
            print('Train Summary | End of Epoch {0} | Time {1:.2f}s | '
                  'Train Loss {2:.3f} | Epoch Accuracy {3:.3f} | Epoch Weighted Recall {4:.3f}'.format(
                epoch + 1, time.time() - start, tr_avg_loss, epoch_acc, epoch_weighted_recall))             # нэг epoch - ын сургалтын үр дүн
            print('-' * 85)


            print('Cross validation...')
            self.model.eval()                        #  Batchnorm болон dropout - ыг унтраан, gradient тооцох явцыг түр зур зогсоож, моделийг өөр өгөгдөл дээр туршина

            val_loss, val_epoch_acc, epoch_weighted_recall = self._run_one_epoch(epoch, cross_valid_mode=True)    # validation  ын үр дүн

            print('-' * 85)
            print('Valid Summary | End of Epoch {0} | Time {1:.2f}s | '
                  'Valid Loss {2:.3f} | Valid Epoch Accuracy {3:.3f} | Epoch weighted recall {4:.3f}'.format(
                epoch + 1, time.time() - start, val_loss, val_epoch_acc, epoch_weighted_recall))
            print('-' * 85)

            if val_loss < self.best_val_loss:           # Хамгийн сайн моделио хадгална.
                self.best_val_loss = val_loss
                save_path = os.path.join(self.file_path, "classification_epoch%d.pth.tar" % (epoch+1))
                torch.save(self.model.state_dict(),save_path)
                print("Илүү сайн модель оллоо, хадгалж байна: %s" % save_path)


    def _run_one_epoch(self, epoch, cross_valid_mode=False):
        start = time.time()
        total_loss = 0

        data_loader = self.tr_loader if not cross_valid_mode else self.cv_loader    # dataloader 
        eval_loader = self.cv_loader.__iter__()
        cross_valid = cross_valid_mode                                               
        epoch_accuracy = 0.0
        epoch_weighted_recall = 0.0
        n_iters = 0

        totalL = len(data_loader)
        for i, (data) in enumerate(data_loader):                # DataLoader - нь iterable обьект учир давталтанд энэ байдлаар ашиглахад л batch болгож, padding хийж, shuffle хийн batch batch аар нь ашиглах боломжтой болгодог
            padded_input, input_lengths, padded_target = data
            if (not cross_valid_mode) & (i % self.print_freq == 0):   # validate хийх үгүйг шийднэ.
                # do eval
                cross_valid = True
                self.model.eval()

            if self.use_cuda:         # Хэрэв gpu ашиглах боломжтой бол tensor утгуудаа gpu руу шилжүүлнэ.
                padded_input = padded_input.cuda()
                input_lengths = input_lengths.cuda()
                padded_target = padded_target.cuda()
            predictions = self.model(padded_input, input_lengths)     # сонгогдсон batch өгөгдөл - с prediction гаргана.
            predictions = predictions.view(-1, predictions.size(-1))  
            loss = self.criterion(predictions, padded_target)         # prediction нь target өгөгдлөөс хэр зөрснийг тооцож, параметруудыг хэр хэмжээгээр өөрчлөхийг шийднэ.
            # print(predictions.shape,padded_target.shape)
            accuracy = float(sum(predictions.argmax(axis=1) == padded_target)) / float(len(padded_target))      # batch өгөгдлөөс хэдийг нь зөв таасныг тооцно.
            if use_cuda:  
              weights = torch.tensor([1 / (float(3 * sum(padded_target == s)) + 0.000001) for s in range(n_classes)]).cuda()
            else:
              weights = torch.tensor([1 / (float(3 * sum(padded_target == s)) + 0.000001) for s in range(n_classes)]).cpu()
            weighted_recall = sum((predictions.argmax(axis=1) == padded_target) * weights[padded_target])    # weight ашиглан илүү бодит recall тооцно (тухайн класст харгалзах утгуудаас хэдийг нь зөв таасныг тооцно, тооцохдоо тухайн класс batch д хэд байгаагаас нь хамааран weight зооно)
            epoch_accuracy += accuracy
            epoch_weighted_recall += weighted_recall
            if not cross_valid:                   
                self.optimizer.zero_grad()      # backpropagation хийхийн өмнө градьентуудаа тэглэнэ.
                loss.backward()                 # моделийн бүх параметрийн алдагдлыг тооцно.
                torch.nn.utils.clip_grad_norm_(self.model.parameters(),self.max_norm)
                self.optimizer.step()           # бүх параметруудаар давтан, градьент утгуудыг нь шинэчилнэ.
            total_loss += loss.item()

            if i % self.print_freq == 0:
                print('Epoch {0} | Iter {1:.4f} | Average Loss {2:.3f} | '
                      'Current Loss {3:.6f} | {4:.1f} ms/batch | {5:.4f} accuracy | '
                      '{6:.3f} weighted recall | {7} class preds'.format(
                    epoch + 1, float(i + 1) / totalL, total_loss / (i + 1),
                    loss.item(), 1000 * (time.time() - start) / (i + 1), accuracy, weighted_recall,   # Сургалтын үед print_freq давтамжаар үзүүлэлтүүдийг хэвлэнэ.
                    [sum(predictions.argmax(axis=1) == padded_target).item()],
                    flush=True))
                if not cross_valid_mode:
                    cross_valid = False
                    self.evali += 1
                    self.model.train()
            self.traini += 1
            n_iters = i

        return total_loss / totalL, epoch_accuracy / n_iters, epoch_weighted_recall / n_iters     # нэг epoch-ын үзүүлэлт


In [ ]:
def clean_line(line):           # тэмдэгт мөр цэвэрлэх функц
    line = line.lower()
    line = re.sub("[\t\n]", " ", line)
    line = re.sub("[<>/@%$^&*\-+()\[\]~\"\';:“”]+", " $ ", line)
    line = re.sub("[0-9]+[., ][0-9]+|[0-9]+", "#", line)
    line = re.sub("( )*([.,?!])", "\g<2> ", line)
    line = re.sub("( )+", " ", line)
    return line

In [ ]:
clean_line("ххэхэ №- хэн  -бэ  ₮:. айн ₮ 45 юу")

'ххэхэ № $ хэн $ бэ ₮ $. айн ₮ # юу'

In [ ]:
def num_param(model):           # моделийн параметруудыг буцаана.
    params = 0
    for p in model.parameters():
        tmp = 1
        for x in p.size():
            tmp *= x
        params += tmp
    return params

In [ ]:
data_file = "/content/drive/MyDrive/ML/news.csv"
train_path = "/content/drive/MyDrive/ML/train.csv"
valid_path = "/content/drive/MyDrive/ML/valid.csv"
save_path = "/content/drive/MyDrive/ML/"
vocab_size = 10000                        # үгсийн сангийн хэмжээ
extra_word_list=["<unk>", "<end>"]        # үгсийн сангийн map - д ашиглагдана. эдгээр нь мөн sequence ын төгсгөл, үгийн санд байхгүй байгаа үгийг тэмдэглэнэ
df = pd.read_csv(data_file, dtype={'news': str, ' label': str}, low_memory=False)    # түүхий датагаа уншина.
df = df.sample(None, 1.0)                                                            # датагаа холино
df = df.dropna()                                                                     # дата дахь хоосон өгөгдөлтэй мөрийг устгана
df = df.iloc[:70000]                                                                 # эхний 70мянган сетийг авна.  
train_data = df.iloc[:int(len(df)*0.8)]                                              # датаныхаа 80хувийг сургалтад, 20 хувийг validation д ашиглана.
valid_data = df.iloc[int(len(df)*0.8):int(len(df)*1)]
print("Train: ",len(train_data)," Valid: ",len(valid_data))
train_data.to_csv(train_path,index=False)                                            # хуваасан датагаа хадгална
valid_data.to_csv(valid_path,index=False)                                                          
print("Loaded file", data_file, "\t#:", len(df))
print("Cleaning lines")

df["news"] = df["news"].apply(lambda txt: clean_line(txt))                           # мэдээтэй баганыг цэвэрлэнэ
words = (" ".join(df["news"].to_list())).split()                                     # бүх үгийг ялгана
vocabs = [items for items, c in Counter(words).most_common()[:vocab_size]]           # хамгийн түгээмэл vocab_size тооны үгийг авна
classes = df[" label"].unique().tolist()                                             # классуудыг list хэлбэрт шилжүүлнэ
del df
n = len(extra_word_list)
vocab = { word.strip(): i + n for i, word in enumerate(vocabs) }                     # хагмийн түгээмэл үгсээр {үг: id} vocab үүсгэнэ        
class_ids = {cls: i for i, cls in enumerate(classes)}                                # класуудаар мөн {class: id} үүсгэнэ
for i, word in enumerate(extra_word_list):
    vocab[word] = i
print("Vocab created")
del words,vocabs

Train:  56000  Valid:  14000
Loaded file /content/drive/MyDrive/ML/news.csv 	#: 70000
Cleaning lines
Vocab created


In [ ]:
num_embeddings = len(vocab) + 1       # embedding хэмжээ
embedding_dim = 256                   # embedding хэмжээс
hidden_size = 512                     # lstm - ын hidden state - ын feature ын тоо. hidden state гэдэг нь lstm ын cell - г хэлнэ.
num_layers = 2                        # lstm - ын recurrent давхрагын тоо
dropout = 0.15                        # dropout хийх магадлал
learning_rate = 0.01                  # optimizer ын learning rate нь хэр weight - ыг өөрчлөхдөө том алхам авахыг хэлнэ. 
epochs = 30                           # хэдэн удаа нийт датанд сургахыг тохирууна
batch_size = 32                       # нэг batch - д багтах input болон target датаны тоо
max_norm = 5                          # gradient clip хийх хамгийн мах утга
print_freq = 5                        # үзүүлэлтүүдийг хэвлэх давтамж
num_workers = 1                       # dataloader iterator үүсэх үед хэдэн ажиллагч процесс үүсэхийг заана
bidirectional = True                  # LSTM хоёр чиглэлтэй үгүйг заана (2 чиглэлтэй бол BiLSTM)
n_classes = len(class_ids)            # классын тоо
use_cuda = torch.cuda.is_available()  # gpu руу хөрвүүлэх боломжтой бол үнэн утга авна
print("use cuda: ", use_cuda)

use cuda:  True


In [ ]:
tr_loader = build_data_loader(train_path, vocab, class_ids, batch_size, num_workers)    # сургалтын dataloader үүсгэх
cv_loader = build_data_loader(valid_path, vocab, class_ids, batch_size, num_workers)    # cross validation dataloader
data = {'tr_loader': tr_loader, 'cv_loader': cv_loader}     

model = LstmClassification(num_embeddings, embedding_dim,hidden_size, num_layers, tr_loader.dataset.num_class, dropout, bidirectional)     # LSTM модель үүсгэх
if use_cuda:
  model.cuda()      # моделио gpu руу шилжүүлэх
print(model)
print("Number of parameters: %d" % num_param(model))      # моделийн параметрүүд
criterion = torch.nn.CrossEntropyLoss(ignore_index=-1, reduction='mean')    # алдаа тооцох criterion
optimizier = torch.optim.Adam(model.parameters(), lr=learning_rate)                     # параметрүүдийн weight өөрчлөх optimizer
solver = Solver(data, model, criterion, optimizier, epochs, max_norm, print_freq,save_path,use_cuda,n_classes)  # сургалтыг удирдах обьект
solver.train()        # сураглтаа эхлүүлцгээе

Loaded Dataset 56000
Loaded Dataset 14000
LstmClassification(
  (embedding): Embedding(10003, 256)
  (lstm): LSTM(256, 512, num_layers=2, batch_first=True, dropout=0.15, bidirectional=True)
  (linearDP): Dropout(p=0.15, inplace=False)
  (fc): Linear(in_features=2048, out_features=9, bias=True)
)
Number of parameters: 12032777
Training...
Epoch 1 | Iter 0.0000 | Average Loss 2.169 | Current Loss 2.168671 | 527.4 ms/batch | 0.1250 accuracy | 0.500 weighted recall | [4] class preds
Epoch 1 | Iter 0.0001 | Average Loss 6.632 | Current Loss 7.184114 | 590.7 ms/batch | 0.0312 accuracy | 0.333 weighted recall | [1] class preds
Epoch 1 | Iter 0.0002 | Average Loss 5.416 | Current Loss 2.888473 | 535.0 ms/batch | 0.0625 accuracy | 0.389 weighted recall | [2] class preds
Epoch 1 | Iter 0.0003 | Average Loss 4.510 | Current Loss 2.350276 | 543.6 ms/batch | 0.1250 accuracy | 0.333 weighted recall | [4] class preds
Epoch 1 | Iter 0.0004 | Average Loss 3.992 | Current Loss 2.335399 | 579.6 ms/batch 

KeyboardInterrupt: ignored

In [ ]:
set("hhe he hheee lol lol lol".split())